<a href="https://colab.research.google.com/github/himoyuzuki/Colab_analytics/blob/main/%E5%90%8D%E5%AD%97%E3%81%8C%E5%8F%8E%E6%9D%9F%E3%81%99%E3%82%8B%E3%83%97%E3%83%AD%E3%82%BB%E3%82%B9%E3%82%92%E5%8F%AF%E8%A6%96%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 名字が収束するプロセスを可視化

生成にはClaude Sonnet 4を活用しました


In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from collections import Counter
import random
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

class SurnameSimulator:
    def __init__(self, initial_surnames, initial_population):
        """
        名字シミュレータの初期化

        Parameters:
        initial_surnames: list - 初期の名字のリスト
        initial_population: int - 初期人口（偶数である必要がある）
        """
        self.initial_surnames = initial_surnames
        self.initial_population = initial_population
        self.reset_simulation()

    def reset_simulation(self):
        """シミュレーションをリセット"""
        # 初期人口を名字でランダムに分配
        self.population = []
        surnames_cycle = (self.initial_surnames * (self.initial_population // len(self.initial_surnames) + 1))[:self.initial_population]
        random.shuffle(surnames_cycle)
        self.population = surnames_cycle

        # 履歴の初期化
        self.generation_history = []
        self.surname_counts_history = []
        self.diversity_history = []
        self.generation = 0

        # 初期状態を記録
        self.record_generation()

    def record_generation(self):
        """現在の世代の状態を記録"""
        surname_counts = Counter(self.population)
        self.generation_history.append(self.generation)
        self.surname_counts_history.append(dict(surname_counts))

        # 多様性指標（シャノン多様性指数）を計算
        total = len(self.population)
        diversity = -sum((count/total) * np.log(count/total) for count in surname_counts.values())
        self.diversity_history.append(diversity)

    def simulate_generation(self):
        """1世代分のシミュレーションを実行"""
        if len(set(self.population)) <= 1:
            return False  # すべて同じ名字になった場合は終了

        # 人口をランダムにペアに分ける
        random.shuffle(self.population)
        new_population = []

        for i in range(0, len(self.population), 2):
            if i + 1 < len(self.population):
                parent1 = self.population[i]
                parent2 = self.population[i + 1]

                # ランダムに片方の名字を選択
                chosen_surname = random.choice([parent1, parent2])

                # 2人の子どもが生まれる
                new_population.extend([chosen_surname, chosen_surname])

        self.population = new_population
        self.generation += 1
        self.record_generation()

        return True

    def run_simulation(self, max_generations=100):
        """指定された世代数までシミュレーションを実行"""
        for _ in range(max_generations):
            if not self.simulate_generation():
                break

    def get_results_dataframe(self):
        """結果をDataFrameとして取得"""
        # 全ての名字を取得
        all_surnames = set()
        for counts in self.surname_counts_history:
            all_surnames.update(counts.keys())

        # DataFrameを作成
        data = []
        for gen, counts in zip(self.generation_history, self.surname_counts_history):
            row = {'Generation': gen}
            for surname in all_surnames:
                row[surname] = counts.get(surname, 0)
            data.append(row)

        return pd.DataFrame(data)

    def create_visualization(self):
        """可視化グラフを作成"""
        df = self.get_results_dataframe()

        # サブプロットを作成
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('名字の人口推移', '名字の種類数推移', '多様性指数推移', '最終世代の名字分布'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "pie"}]]
        )

        # 1. 名字の人口推移
        surname_cols = [col for col in df.columns if col != 'Generation']
        colors = px.colors.qualitative.Set3

        for i, surname in enumerate(surname_cols):
            if df[surname].sum() > 0:  # 人口が0でない名字のみ表示
                fig.add_trace(
                    go.Scatter(
                        x=df['Generation'],
                        y=df[surname],
                        name=surname,
                        line=dict(color=colors[i % len(colors)]),
                        stackgroup='one'
                    ),
                    row=1, col=1
                )

        # 2. 名字の種類数推移
        surname_variety = [sum(1 for count in counts.values() if count > 0)
                          for counts in self.surname_counts_history]

        fig.add_trace(
            go.Scatter(
                x=self.generation_history,
                y=surname_variety,
                name='名字の種類数',
                line=dict(color='red', width=3)
            ),
            row=1, col=2
        )

        # 3. 多様性指数推移
        fig.add_trace(
            go.Scatter(
                x=self.generation_history,
                y=self.diversity_history,
                name='シャノン多様性指数',
                line=dict(color='blue', width=3)
            ),
            row=2, col=1
        )

        # 4. 最終世代の名字分布（円グラフ）
        final_counts = self.surname_counts_history[-1]
        final_surnames = list(final_counts.keys())
        final_values = list(final_counts.values())

        fig.add_trace(
            go.Pie(
                labels=final_surnames,
                values=final_values,
                name="最終分布"
            ),
            row=2, col=2
        )

        # レイアウトの設定
        fig.update_layout(
            height=800,
            title_text=f"名字継承シミュレーション結果（{self.generation}世代）",
            showlegend=True
        )

        # 軸ラベルの設定
        fig.update_xaxes(title_text="世代", row=1, col=1)
        fig.update_yaxes(title_text="人口", row=1, col=1)
        fig.update_xaxes(title_text="世代", row=1, col=2)
        fig.update_yaxes(title_text="名字の種類数", row=1, col=2)
        fig.update_xaxes(title_text="世代", row=2, col=1)
        fig.update_yaxes(title_text="多様性指数", row=2, col=1)

        return fig

# インタラクティブなシミュレーション関数
def run_interactive_simulation(surnames_input="田中,佐藤,鈴木,高橋,伊藤,渡辺,山本,中村,小林,加藤",
                             population=100,
                             max_generations=50,
                             random_seed=42):
    """
    インタラクティブなシミュレーション実行

    Parameters:
    surnames_input: str - カンマ区切りの名字リスト
    population: int - 初期人口
    max_generations: int - 最大世代数
    random_seed: int - ランダムシード
    """
    # ランダムシードを設定
    random.seed(random_seed)
    np.random.seed(random_seed)

    # 名字リストを作成
    surnames = [s.strip() for s in surnames_input.split(',')]

    # 人口が偶数でない場合は調整
    if population % 2 != 0:
        population += 1

    print(f"シミュレーション設定:")
    print(f"初期名字: {surnames}")
    print(f"初期人口: {population}人")
    print(f"最大世代数: {max_generations}")
    print(f"ランダムシード: {random_seed}")
    print("-" * 50)

    # シミュレータを作成・実行
    simulator = SurnameSimulator(surnames, population)
    simulator.run_simulation(max_generations)

    # 結果を表示
    final_surnames = list(simulator.surname_counts_history[-1].keys())
    final_diversity = simulator.diversity_history[-1]

    print(f"\nシミュレーション結果:")
    print(f"実行世代数: {simulator.generation}")
    print(f"残存名字数: {len(final_surnames)}")
    print(f"残存名字: {final_surnames}")
    print(f"最終多様性指数: {final_diversity:.3f}")

    # グラフを表示
    fig = simulator.create_visualization()
    fig.show()

    return simulator

# ウィジェットを使用したインタラクティブインターフェース
def create_interactive_interface():
    """インタラクティブなインターフェースを作成"""

    surnames_widget = widgets.Text(
        value="田中,佐藤,鈴木,高橋,伊藤,渡辺,山本,中村,小林,加藤",
        description='初期名字:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )

    population_widget = widgets.IntSlider(
        value=100,
        min=20,
        max=500,
        step=10,
        description='初期人口:',
        style={'description_width': 'initial'}
    )

    generations_widget = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=10,
        description='最大世代数:',
        style={'description_width': 'initial'}
    )

    seed_widget = widgets.IntText(
        value=42,
        description='ランダムシード:',
        style={'description_width': 'initial'}
    )

    # インタラクティブウィジェットを作成
    interactive_plot = interactive(
        run_interactive_simulation,
        surnames_input=surnames_widget,
        population=population_widget,
        max_generations=generations_widget,
        random_seed=seed_widget
    )

    return interactive_plot

# 使用例
if __name__ == "__main__":
    print("名字継承シミュレータ")
    print("=" * 50)
    print("このシミュレータは、婚姻による名字の継承を通じて、")
    print("世代を重ねるごとに名字の多様性が失われる様子を可視化します。")
    print("\n以下の関数を使用してシミュレーションを実行できます：")
    print("\n1. run_interactive_simulation() - 基本的なシミュレーション実行")
    print("2. create_interactive_interface() - インタラクティブなウィジェット版")
    print("\n例：")
    print("# 基本実行")
    print("simulator = run_interactive_simulation()")
    print("\n# インタラクティブ版")
    print("interface = create_interactive_interface()")
    print("display(interface)")

名字継承シミュレータ
このシミュレータは、婚姻による名字の継承を通じて、
世代を重ねるごとに名字の多様性が失われる様子を可視化します。

以下の関数を使用してシミュレーションを実行できます：

1. run_interactive_simulation() - 基本的なシミュレーション実行
2. create_interactive_interface() - インタラクティブなウィジェット版

例：
# 基本実行
simulator = run_interactive_simulation()

# インタラクティブ版
interface = create_interactive_interface()
display(interface)


In [2]:
simulator = run_interactive_simulation(population=1000, max_generations=500)

シミュレーション設定:
初期名字: ['田中', '佐藤', '鈴木', '高橋', '伊藤', '渡辺', '山本', '中村', '小林', '加藤']
初期人口: 1000人
最大世代数: 500
ランダムシード: 42
--------------------------------------------------

シミュレーション結果:
実行世代数: 500
残存名字数: 4
残存名字: ['伊藤', '小林', '鈴木', '田中']
最終多様性指数: 1.003
